In [5]:
import torch
import torch.nn as nn
from torch.utils.data import dataset
from torchvision import transforms


In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip -O /content/drive/MyDrive/data/testSet.zip

In [ ]:
!unzip /content/drive/MyDrive/data/testSet.zip -d /content/drive/MyDrive/data/

In [ ]:
!git clone https://github.com/1-1is0/deep-projects

In [ ]:
!cp /content/deep-projects/q2/*.py .

In [15]:
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
from dataloader import load_data, ImageDataset, Rescale, ToTensor, GetAlexConv
path = "/content/drive/MyDrive/data/tiny-imagenet-200/train/*"
test_df, train_df = load_data(path)
train_dataset = ImageDataset(
    train_df,
    transform=transforms.Compose(
        [
            Rescale((227, 227)),
            ToTensor(),
        ]
    ),
    alexnet_feature=GetAlexConv(layer_number=[0, 5], layer_name="maxpool2"),
)
test_dataset = ImageDataset(
    test_df,
    transform=transforms.Compose(
        [
            Rescale((227, 227)),
            ToTensor(),
        ]
    ),
    alexnet_feature=GetAlexConv(layer_number=[0, 5], layer_name="maxpool2"),
)


trainloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=0
)
testloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True, num_workers=0
)

print("train_dataset:", len(train_dataset))
print("test_dataset", len(test_dataset))


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
from model import AlexConv2
net = AlexConv2().to(device)

print(net)


In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

scheduler = ExponentialLR(optimizer, gamma=0.9)


In [ ]:

from model import train_model
dataloader = {
    "train": trainloader,
    "val": testloader,
}
dataset_size = {
    "train": len(train_dataset),
    "val": len(test_dataset),
}

res = train_model(data_loader=dataloader, model=net, criterion=criterion, optimizer=optimizer, scheduler=scheduler, device=device, epochs=1, dataset_size=dataset_size)
